# Tensorflow HW

Using the DNN regressor in TF, construct a Neural Network capable of finding the volume of a cylinder given a radius (r) and (h). Simulate the data, which will be comprised of r,h, and V. Generate more data if the results are poor as measure by RMSE.

In [77]:
import tensorflow as tf
from tensorflow.estimator import DNNRegressor 
from tensorflow.estimator.inputs import numpy_input_fn 
from sklearn import preprocessing, model_selection, metrics
from tensorflow.python.training import optimizer
import pandas as pd
import random
import numpy as np

## Data preparation

In [78]:
# generate the data and added some noise to it. + np.random.rand(999)
radius =  np.random.uniform(1, 100, 1000000) 
height = np.random.uniform(1, 100, 1000000) 
volume = np.pi * radius**2 * height

In [79]:
X = pd.DataFrame({"radius": radius, "height": height})
y = volume

In [80]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25, random_state=42)

In [81]:
scaler = preprocessing.StandardScaler()
x_train = scaler.fit_transform(x_train)

In [82]:
#machine learning models take numbers as input and outputs numbers for efficient implementation. 
#In Tensorflow, the models take Dense Tensors as input and outputs Dense Tensors\
#we only get numeric column here
feature_columns = [tf.feature_column.numeric_column('x', shape=np.array(x_train).shape[1:])]

## Training model

In [88]:
train_input_fn = numpy_input_fn(
    x={'x': x_train}, 
    y=y_train,     
    batch_size=128,
    num_epochs=1,
    shuffle=True,
    queue_capacity=1000,
    num_threads=1)
#returns input function that would feed dict of numpy arrays into the model.


regressor = DNNRegressor(feature_columns=feature_columns, hidden_units=[1024, 512, 256],
                        optimizer=tf.train.AdagradOptimizer(learning_rate=0.05)   #添加 optimizer
)
                         
regressor.train(input_fn=train_input_fn, steps=20000) 

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/x2/rwr8_g2x0_v7z00l90vswnwc0000gn/T/tmp4vzh3lpf', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12e1366d8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create Chec

## Predicting 

In [89]:
x_transformed = scaler.transform(x_test)
test_input_fn = numpy_input_fn(x={'x': x_transformed}, y=y_test, num_epochs=1, shuffle=False)
predictions = regressor.predict(input_fn=test_input_fn)
y_predicted = np.array(list(p['predictions'] for p in predictions))
y_predicted = y_predicted.reshape(np.array(y_test).shape)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/x2/rwr8_g2x0_v7z00l90vswnwc0000gn/T/tmp4vzh3lpf/model.ckpt-5860
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


## Evaluation

In [90]:
regressor.evaluate(input_fn = test_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-28-02:50:07
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/x2/rwr8_g2x0_v7z00l90vswnwc0000gn/T/tmp4vzh3lpf/model.ckpt-5860
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-28-02:50:12
INFO:tensorflow:Saving dict for global step 5860: average_loss = 395605250.0, global_step = 5860, label/mean = 533685.6, loss = 50614796000.0, prediction/mean = 534677.8
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5860: /var/folders/x2/rwr8_g2x0_v7z00l90vswnwc0000gn/T/tmp4vzh3lpf/model.ckpt-5860


{'average_loss': 395605250.0,
 'global_step': 5860,
 'label/mean': 533685.6,
 'loss': 50614796000.0,
 'prediction/mean': 534677.8}

In [91]:
score_sklearn = metrics.mean_squared_error(y_predicted, y_test)
print('RMSE = {0:f}'.format(score_sklearn**(0.5)))

RMSE = 19889.834221
